In [58]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# if print results in 'cuda' means GPU can be used to process parallel processes

block_size = 8
batch_size = 4
max_iters = 50000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 5000

cuda


In [59]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# All the chars in the text
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [60]:
#Tokenizer

string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])

In [61]:
#Train and evaluate results parameters

n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)


inputs:
tensor([[58,  1, 68, 59,  1, 46, 68, 58],
        [72, 72, 62, 68, 67,  1, 66, 68],
        [58, 57,  1, 73, 61, 58,  1, 67],
        [72, 64, 58, 57,  1, 73, 61, 58]], device='cuda:0')
targets:
tensor([[ 1, 68, 59,  1, 46, 68, 58,  1],
        [72, 62, 68, 67,  1, 66, 68, 75],
        [57,  1, 73, 61, 58,  1, 67, 62],
        [64, 58, 57,  1, 73, 61, 58,  1]], device='cuda:0')


In [62]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [63]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


18﻿;)mv[zGAxD!P'.9)gUe;nzqR&Dk.u RK3729i1B4YD_,zz)7Di.ywSYynuxgoP6vyZYkp&z3Jjmv0('1-.siaTu-]9oSb*﻿P")G0J&W2LR;BjX
tH4c_Bu﻿ iVJVNeqk)t(Odd?hN[hLoSQIpcazo!"ZSrDVb6PrO]?'VKsx3Qx[k32"))W2680Kkix7[Jme N?r48r:lZ,?0qeCPsbVnOH5"EyWnxp;F9oJyP.[0:bnWw﻿ynqMmz)7j?bR':au?H!fSf-zkS.zuysiv8B!mv,r]u i!- ?DAa]*VT"SfVX2j*B-a LknO?rD5*d;Jg4gB3C'2Gi]_E,Dwn.-hdhf;6SJ6]hpK﻿Gq&U!Z*tpX:&Xb*a zsjv﻿).
'kA&8rW-a5kZ"_&AwZE,3nb3oSv;g
5kPpSc&
Uu[S7j*,yn6T1-MS"caktN:NS;nNUSKjb4X"gSb[e43DAA4qd.z8Jt,qa&kkPgQ*,Fs:Sz"*UZX2"F7zH
E


In [64]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.883, val loss: 4.875
step: 5000, train loss: 3.819, val loss: 3.823
step: 10000, train loss: 3.166, val loss: 3.181
step: 15000, train loss: 2.819, val loss: 2.842
step: 20000, train loss: 2.649, val loss: 2.676
step: 25000, train loss: 2.564, val loss: 2.594
step: 30000, train loss: 2.514, val loss: 2.555
step: 35000, train loss: 2.489, val loss: 2.524
step: 40000, train loss: 2.471, val loss: 2.510
step: 45000, train loss: 2.458, val loss: 2.504
2.3776614665985107


In [65]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


 iguar?"
ly nis barmail ug t Wo?" s, t.
"


of co oud t. EBMreando ld."
"Thed d ased wero mytheswr asaf th w se n wofomo s h te ceald ce.
e Pin, agor.'s Moy ye f trsanthembe'he t scis ny,2" othe w y, h  f aloon ahen emerld heghe tt.
pomo whedino thend; fo
ndord ched tscouanu
t hele is, bed bit w im y as gesend otho wim-ad cre

he y ebun mst MO jem.
[0Gok YRASwothasee is, ath hove An " tr ay  aveme Ljt t thir and d sounowinlofo he t ary ar b, ul  WUut "Inu ane chan th."in'thrcangimo hanus he ched
